<a href="https://colab.research.google.com/github/marytheresa/API/blob/master/Omdena_Back_Pain_fuse_files_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import pandas as pd
import polars as pl
import tensorflow as tf
import gc

from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Specify the folder path where the .parquet files are located
folder_path = '/content/drive/MyDrive/Colab Notebooks/Omdena/datasets/02 - intermediary'

# Get a list of .parquet file paths in the folder
file_paths = glob.glob(folder_path + '/*.parquet')

# Define batch size for processing files
batch_size = 1

# Loop over the file paths and process files in batches
count = 0
for i in range(0, len(file_paths), batch_size):
    batch_file_paths = file_paths[i:i+batch_size]

    # Process each file in the batch
    for file_path in batch_file_paths:
        count += 1
        df = pl.read_parquet(file_path)
        if '__index_level_0__' in df.columns:
            df = df.drop('__index_level_0__')

        df = df.with_columns(pl.col("t").cast(pl.Utf8))

        df = df.with_columns(
              pl.col('t').str.strptime(pl.Date, format='%m/%d/%Y %H:%M:%S', strict=False)
    )

        # Perform operations on the DataFrame
        print(df.head(1))

        # Print the columns in the DataFrame
        print(f"Columns in file {file_path}:")
        print(df.columns)
        print(df.shape)
        print(df.dtypes)

        # Free up memory by deleting the DataFrame
        del df

        # Invoke garbage collection
        gc.collect()

print("Total number of files:", count)



Mounted at /content/drive
shape: (1, 5)
┌──────┬───────────────────────┬──────────┬───────────────────────┬────────────────────────────────┐
│ t    ┆ logger_name           ┆ priority ┆ message               ┆ user_id                        │
│ ---  ┆ ---                   ┆ ---      ┆ ---                   ┆ ---                            │
│ date ┆ str                   ┆ str      ┆ str                   ┆ str                            │
╞══════╪═══════════════════════╪══════════╪═══════════════════════╪════════════════════════════════╡
│ null ┆  data handler wrapper ┆  INFO    ┆ starting data handler ┆ 3b12d290-5ca7-43ae-9199-747d9b │
│      ┆                       ┆          ┆ wrapper i…            ┆ fe…                            │
└──────┴───────────────────────┴──────────┴───────────────────────┴────────────────────────────────┘
Columns in file /content/drive/MyDrive/Colab Notebooks/Omdena/datasets/02 - intermediary/Prashanth-log.parquet:
['t', 'logger_name', 'priority', 'messag

In [2]:
import os
import pandas as pd
import polars as pl
#from polars.lazy import *
import pyarrow.parquet as pq

from google.colab import drive

In [3]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Specify the folder path where the .parquet files are located
folder_path = '/content/drive/MyDrive/Colab Notebooks/Omdena/datasets/02 - intermediary'

# Get a list of file paths for Parquet files ending with "-log.parquet"
log_file_paths = [file for file in os.listdir(folder_path) if file.endswith('-log.parquet')]

# Get a list of file paths for Parquet files ending with "-raw.parquet"
raw_file_paths = [file for file in os.listdir(folder_path) if file.endswith('-raw.parquet')]

# Get a list of file paths for Parquet files ending with "-score.parquet"
score_file_paths = [file for file in os.listdir(folder_path) if file.endswith('-score.parquet')]

# Read and append each Parquet file to the list
def MergeFiles(file_paths):
  dfs = []
  for file_path in file_paths:
      full_path = os.path.join(folder_path, file_path)
      df = pl.read_parquet(full_path)
      dfs.append(df)
  return dfs

Mounted at /content/drive


In [ ]:
# Concatenate the Log DataFrames row-wise
log_df = pl.DataFrame([])
count = 0
for df in MergeFiles(log_file_paths):
  print(df.columns)
  count = count + 1
  print(count)

  # Read the raw file as a Polars DataFrame
  if '__index_level_0__' in df.columns:
    df = df.drop('__index_level_0__')

  df = df.with_columns(pl.col("t").cast(pl.Utf8))
  df = df.with_columns(pl.col("t").str.slice(0, 19))
  df = df.with_columns(pl.col('t').str.strptime(pl.Datetime, format='%Y-%m-%d %H:%M:%S', strict=False))

  log_df = log_df.vstack(df)

# Save the concatenated DataFrame to a new Parquet file
output_path = '/content/drive/MyDrive/Colab Notebooks/Omdena/fused_files_backup/merged_log.parquet'
log_df.write_parquet(output_path)

print(f"Concatenated DataFrame saved to {output_path}")

['t', 'logger_name', 'priority', 'message', 'user_id', '__index_level_0__']
1
['t', 'logger_name', 'priority', 'message', 'user_id', '__index_level_0__']
2
['t', 'logger_name', 'priority', 'message', 'user_id', '__index_level_0__']
3
['t', 'logger_name', 'priority', 'message', 'user_id']
4
['t', 'logger_name', 'priority', 'message', 'user_id', '__index_level_0__']
5
['t', 'logger_name', 'priority', 'message', 'user_id']
6
Concatenated DataFrame saved to /content/drive/MyDrive/Colab Notebooks/Omdena/fused_files_backup/merged_log.parquet


In [4]:
# Set the batch size
batch_size = 10000000

# Initialize an empty DataFrame to store the concatenated data
raw_df = pl.DataFrame([])

# Iterate over the raw file paths
for file_path in raw_file_paths:
    # Read the raw file as a Polars DataFrame
    full_path = os.path.join(folder_path, file_path)
    df = pl.read_parquet(full_path)
    if '__index_level_0__' in df.columns:
        df = df.drop('__index_level_0__')

    df = df.with_columns(pl.col("t").cast(pl.Utf8))
    df = df.with_columns(pl.col("t").str.slice(0, 19))
    df = df.with_columns(pl.col('t').str.strptime(pl.Datetime, format='%Y-%m-%d %H:%M:%S', strict=False))
    df = df.with_columns(pl.col("left", "right", "acc", "gyro").cast(pl.List(pl.Int32)))
    df = df.with_columns(pl.col("v").cast(pl.Int32))

    # Process and concatenate the DataFrames in batches
    num_batches = len(df) // batch_size + 1  # Calculate the number of batches

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size

        # Extract the batch from the DataFrame
        batch = df[start_idx:end_idx]

        # Concatenate the batch DataFrame with the main DataFrame
        raw_df = raw_df.vstack(batch)

# Save the concatenated DataFrame to a new Parquet file
output_path = '/content/drive/MyDrive/Colab Notebooks/Omdena/fused_files_backup/merged_raw.parquet'
raw_df.write_parquet(output_path)

print(f"Concatenated DataFrame saved to {output_path}")

Concatenated DataFrame saved to /content/drive/MyDrive/Colab Notebooks/Omdena/fused_files_backup/merged_raw.parquet


In [ ]:
# Concatenate the Score DataFrames row-wise
score_df = pl.DataFrame([])
for df in MergeFiles(score_file_paths):

  # Read the raw file as a Polars DataFrame
  if '__index_level_0__' in df.columns:
    df = df.drop('__index_level_0__')

  df = df.with_columns(pl.col("t").cast(pl.Utf8))
  df = df.with_columns(pl.col("t").str.slice(0, 19))
  df = df.with_columns(pl.col('t').str.strptime(pl.Datetime, format='%Y-%m-%d %H:%M:%S', strict=False))

  score_df = score_df.vstack(df)

# Save the concatenated DataFrame to a new Parquet file
output_path = '/content/drive/MyDrive/Colab Notebooks/Omdena/fused_files_backup/merged_score.parquet'
score_df.write_parquet(output_path)

print(f"Concatenated DataFrame saved to {output_path}")


Concatenated DataFrame saved to /content/drive/MyDrive/Colab Notebooks/Omdena/fused_files_backup/merged_score.parquet


In [1]:
!pip install gitpython
from getpass import getpass
import os
from git import Repo

# Clone the repository to Colab
repo_url = getpass('Enter the GitHub repository URL: ')
repo_path = os.path.join(os.getcwd(), 'repository')
Repo.clone_from(repo_url, repo_path)

# Change the working directory to the repository folder
os.chdir(repo_path)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
Enter the GitHub repository URL: ··········


In [2]:
!mkdir new-folder

In [27]:
import shutil

# Specify the file path and destination folder
file_path = "/content/repository/Omdena_Back Pain_fuse files_script.ipynb"
destination_folder = "/content/repository/new-folder"

# Move the file to the destination folder
shutil.move(file_path, destination_folder)



FileNotFoundError: ignored

In [ ]:
/content/repository

In [24]:
os.getcwd()

'/content/repository'

In [25]:
!ls

data.dvc  new-folder  requirements.txt	src


In [26]:
# Check the current working directory explicitly
%pwd

'/content/repository'

In [ ]:
mv /content/repository/Omdena_Back Pain_fuse files_script.py new-folder/
